In [1]:
import cv2
import requests
import io
import json
import os
import random
from tqdm import tqdm
import shutil


# ocr 이전 파일 위치
src = '../data/telemoji/MTCNN/train/disgust/'    # *수정 필요*   
# ocr 이후 파일 위치
to_dir = '../data/telemoji/OCR/train/disgust/disgust_filter/'   # 이미지 # *수정 필요*
char_dir = '../data/telemoji/OCR/train/disgust/character/'   # 글자

url_api = "https://api.ocr.space/parse/image"
files_ls = os.listdir(src)

In [ ]:
file_path = os.path.join(src, '1376d0360b9856299a9ad4bea70b4760369ec77eb2df113f9871d67f.jpg')
img = cv2.imread(file_path) 
_, compressedimage = cv2.imencode(".jpg", img)  # (Boolean, ndarray)
                                                # memory buffer에 이미지 넣기
file_bytes = io.BytesIO(compressedimage)        # memory buffer에 있는 byte 형태 데이터 

result = requests.post(url_api,
                       files={file_path: file_bytes},
                       data={"apikey": "78c8df23e688957", # *수정 필요*
                             "language": "eng"})
result = result.content.decode() 
result = json.loads(result)
print(result)

In [2]:
# Ocr
for file in tqdm(files_ls): 
    try:
        file_path = os.path.join(src, file)
        img = cv2.imread(file_path) 
        _, compressedimage = cv2.imencode(".jpg", img)  # (Boolean, ndarray)
                                                        # memory buffer에 이미지 넣기
        file_bytes = io.BytesIO(compressedimage)        # memory buffer에 있는 byte 형태 데이터 

        result = requests.post(url_api,
                               files={file_path: file_bytes},
                               data={"apikey": "78c8df23e688957", # *수정 필요*
                                     "language": "eng"})
        result = result.content.decode() 
        result = json.loads(result)
        parsed_results = result.get("ParsedResults")[0]
        text_detected = parsed_results.get("ParsedText")
        if bool(text_detected) == True:
            print(text_detected)
            shutil.move(file_path, char_dir + file)        # char_dir로 파일 이동
        else:
            shutil.move(file_path, to_dir + file)          # to_dir로 파일 이동
    except (AttributeError, IndexError, TypeError) as e:
        print(e)
        print(file)
        break

  0%|                                                                                         | 0/3020 [00:00<?, ?it/s]

'str' object has no attribute 'get'
267a099884c08390fc336ad139ac6c0c0e8436b7ccc864d76e0349aa.jpg
